# CSS 120: Environmental Data Science

## Climatic Reanalysis

### Umberto Mignozzetti (UCSD)

(Based on Pythia and ClimateMatch)

## Today's Class

The ocean's motion is driven by radiation from the sun, winds, and various sources & sinks of fresh water (precipitation, rivers, melting and freezing ice).

We will study two processes today:

1. In the surface, winds drags on the surface of the ocean which results in ocean transport, known as Ekman transport.

2. In larger scale, ocean movement driven by these density differences is known as the *thermohaline circulation*. 
    + The density of ocean water is influenced by temperature (thermo) and salinity (haline), and fluid motion occur in response to pressure gradients caused by these density variations.<br><br>


In the next lecture, we will explore the ocean's vast heat capacity, which has a significant impact on the climate system.
    
We will study the [ECCO (Estimating the Circulation and Climate of the Ocean)](https://www.ecco-group.org) reanalysis data for studying these processes.

## Packages

In [ ]:
# Some installs (if needed)

# !pip install cmocean
# !pip install gsw

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

## Packages

In [ ]:
from intake import open_catalog
import os
import pooch
import tempfile
import numpy as np
import xarray as xr
import cmocean
import gsw
from scipy import integrate

## Packages

In [ ]:
# Packages
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import cartopy as cart
from cartopy import crs as ccrs, feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

##  Helper function

(data pooch)

In [ ]:
# Helper function
def pooch_load(filelocation = None, filename = None, processor = None):
    shared_location = "~/"; user_temp_cache = tempfile.gettempdir()
    if os.path.exists(os.path.join(shared_location, filename)):
        file = os.path.join(shared_location, filename)
    else:
        file = pooch.retrieve(filelocation, known_hash = None, fname = os.path.join(user_temp_cache, filename), processor = processor)
    return file

## Loading Ocean and Atmosphere Reanalysis Data

We will load atmospheric near-surface winds (at 10-meter height), and then load the oceanic surface currents from ECCO reanalysis data

*Note, each of these variables is a velocity with two components (zonal and meridional). These two velocity components must be loaded separately for each variable, so you will load four datasets.*

## Atmospheric Wind

Wind in east/west direction labeled here as $u$

In [ ]:
fname_atm_wind_u = "wind_evel_monthly_2016.nc"
url_atm_wind_u = "https://osf.io/ke9yp/download"
atm_wind_u = xr.open_dataarray(pooch_load(url_atm_wind_u, fname_atm_wind_u))
atm_wind_u

## Atmospheric Wind

Wind in north/south direction labeled here as $v$

In [ ]:

fname_atm_wind_v = "wind_nvel_monthly_2016.nc"
url_atm_wind_v = "https://osf.io/9zkgd/download"
atm_wind_v = xr.open_dataarray(pooch_load(url_atm_wind_v, fname_atm_wind_v))
atm_wind_v

## Atmospheric Wind

Oceanic surface current from ECCO ($u$ component)

In [ ]:
fname_ocn_surface_current_u = "evel_monthly_2016.nc"
ocn_surface_current_u = xr.open_dataarray(fname_ocn_surface_current_u)
ocn_surface_current_u

## Atmospheric Wind

Oceanic surface current from ECCO ($v$ component)

In [ ]:
# current in east/west direction labeled here as 'v'
fname_ocn_surface_current_v = "nvel_monthly_2016.nc"
url_ocn_surface_current_v = "https://osf.io/download/vzdn4/"
ocn_surface_current_v = xr.open_dataarray(
    pooch_load(url_ocn_surface_current_v, fname_ocn_surface_current_v)
)
ocn_surface_current_v

## Exploring the Reanalysis Data

Let's examine the time (or temporal/output) frequency, which descibes the rate at which the reanalysis data is provided, for one of the ECCO variables (*atm_wind_u*). 

*Note that all the variables should have the same output frequency.*

In [ ]:
atm_wind_u.time

## Plotting the Annual Mean of Global Surface Wind Stress

In this section, we will create global maps displaying the annual mean of atmospheric 10m winds.

First, we should compute the annual mean of the surface wind variables. 

We do so by averaging over the time dimension using `.mean(dim='time')`. 

Since you have monthly data spanning only one year, `.mean(dim='time')` will give the annual mean for the year 2016.

## Plotting the Annual Mean of Global Surface Wind Stress

Annual mean of the surface wind variables.

In [ ]:
# compute the annual mean of atm_wind_u
atm_wind_u_an_mean = atm_wind_u.mean(dim="time")
atm_wind_u_an_mean

## Plotting the Annual Mean of Global Surface Wind Stress

Annual mean of the surface wind variables.

In [ ]:
# take the annual mean of atm_wind_stress_v
atm_wind_v_an_mean = atm_wind_v.mean(dim="time")
atm_wind_v_an_mean

## Plotting the Annual Mean of Global Surface Wind Stress

We currently have seperate zonal and meridional wind velocity components $(u,v)$. 

An effective way of visualizing the total surface wind stress is to create a global map of the *magnitude* and *direction* of the wind velocity vector. 

This type of plot is known as a vector field. 

A [vector](https://glossary.ametsoc.org/wiki/Vector) is a special mathematical quantity that has both magnitude and direction, just like the wind! 

The velocity components describe the intensity of wind blowing in the zonal ($u$) or meridional ($v$) directions. 

Specifically, wind can blow eastward (positive $u$) or westward (negative $u$), as well as northward (positive $v$) or southward (negative $v$).

## Plotting the Annual Mean of Global Surface Wind Stress

The total velocity vector is the *vector sum* of these two components and exhibits varying magnitude and direction. 

The magnitude ($||u||$) is:

\begin{align}
||u|| = \sqrt{u^2 + v^2},  \ \  \ \ 
\end{align}

The direction ($\theta$) is:

$$
\theta = tan^{-1}\bigg(\frac{v}{u}\bigg)
$$

When plotting a vector field using a computer, it is commonly referred to as a quiver plot. 

In our case, we will utilize a [quiver function created by Ryan Abernathey](https://rabernat.github.io/intro_to_physical_oceanography/07_ekman.html) that calculates the magnitude and direction of the total velocity vector based on the given zonal and meridional components.

We will overlay the quiver plot on top of the annual mean ocean surface temperature (labeled here as theta).

In [ ]:
fname_surface_temp = "surface_theta.nc"
url_fname_surface_temp = "https://osf.io/98ksr/download"
ocn_surface_temp = xr.open_dataarray(
    pooch.retrieve(url_fname_surface_temp, known_hash=None, fname=fname_surface_temp)
)
ocn_surface_temp

## Plotting the Annual Mean of Global Surface Wind Stress

In [ ]:
# Longitude and latitude coordinates for plotting
lon = atm_wind_u_an_mean.longitude
lat = atm_wind_u_an_mean.latitude

# Magnitude of total velocity
mag = (atm_wind_u_an_mean**2 + atm_wind_v_an_mean**2) ** 0.5

# Coarsen the grid so the arrows are distinguishable by only selecting
# some longitudes and latitudes defined by sampling_x and sampling_y.
slx = slice(None, None, 20)
sly = slice(None, None, 20)
sl2d = (sly, slx)

## Plotting the Annual Mean of Global Surface Wind Stress

In [ ]:
# Figure
fig, ax = plt.subplots(
    figsize=(12, 6), subplot_kw={"projection": ccrs.Robinson(central_longitude=180)}
)

c = ax.contourf(lon, lat, ocn_surface_temp, alpha=0.5)

# plot quiver arrows indicating vector direction (winds are in blue, alpha is for opacity)
q = ax.quiver(lon[slx], lat[sly], atm_wind_u_an_mean[sl2d], 
              atm_wind_v_an_mean[sl2d], color="b", alpha=0.5)

ax.quiverkey(q, 175, 95, 5, "5 m/s", coordinates="data")

fig.colorbar(c, label="Sea Surface Temperature (degC)")

## Comparing Global Maps of Surface Currents and Winds

Now, let us compute the annual mean of the ocean surface currents, similar to your above analyses of atmospheric winds, and create a global map that shows both of these variables simultaneously.

In [ ]:
# take the annual mean of ocn_surface_current_u
ocn_surface_current_u_an_mean = ocn_surface_current_u.mean(dim="time")
ocn_surface_current_u_an_mean

## Comparing Global Maps of Surface Currents and Winds

In [ ]:
# take the annual mean of ocn_surface_current_v
ocn_surface_current_v_an_mean = ocn_surface_current_v.mean(dim="time")
ocn_surface_current_v_an_mean

## Comparing Global Maps of Surface Currents and Winds

Let's add ocean surface currents to the previous plot above, using **red** quivers. Note the scale of the arrows for the ocean and atmosphere are very different.

In [ ]:
# longitude ad latitude coordinates for plotting
lon = atm_wind_u_an_mean.longitude
lat = atm_wind_u_an_mean.latitude

# calculate magnitude of total velocity
mag = (atm_wind_u_an_mean**2 + atm_wind_v_an_mean**2) ** 0.5

# coarsen the grid so the arrows are distinguishable by only selecting
# some longitudes and latitudes defined by sampling_x and sampling_y.
slx = slice(None, None, 20)
sly = slice(None, None, 20)
sl2d = (sly, slx)

## Comparing Global Maps of Surface Currents and Winds

In [ ]:
# fig, ax = plt.subplots(**kwargs)
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={"projection": ccrs.Robinson(central_longitude=180)})

c = ax.contourf(lon, lat, ocn_surface_temp, alpha=0.5)

# plot quiver arrows indicating vector direction (winds are in blue, alpha is for opacity)
q1 = ax.quiver(lon[slx],lat[sly],atm_wind_u_an_mean[sl2d],
               atm_wind_v_an_mean[sl2d], color="b", alpha=0.5)

# plot quiver arrows indicating vector direction (ocean currents are in red, alpha is for opacity)
q2 = ax.quiver(lon[slx], lat[sly], ocn_surface_current_u_an_mean[sl2d],
               ocn_surface_current_v_an_mean[sl2d], color="r", alpha=0.5)

ax.quiverkey(q1, 175, 95, 5, "5 m/s", coordinates="data")

ax.quiverkey(q2, 150, 95, 0.05, "5 cm/s", coordinates="data")

fig.colorbar(c, label="Sea Surface Temperature (degC)")

## Currents and wind direction

You may notice that the surface currents (red) are typically not aligned with the wind direction (blue). In fact, the surface ocean currents flow at an angle of approximately 45 degrees to the wind direction! 

The combination of [Coriolis force](https://en.wikipedia.org/wiki/Coriolis_force) and the friction between ocean layers causes the wind-driven currents to turn and weaken with depth, eventually dissapearing entirely at depth. 

The resulting current *profile* is called the **[Ekman Spiral](https://en.wikipedia.org/wiki/Ekman_spiral)**, and the depth over which the spiral is present is called the **Ekman layer**.

While the shape of this spiral can vary in time and space, the depth-integrated transport of water within the Ekman layer is called [**Ekman transport**](https://en.wikipedia.org/wiki/Ekman_transport). 

Ekman transport is always *90 degrees to the right* of the wind in the Northern Hemisphere, and 90 degrees to the *left* of the wind in the Southern Hemisphere. 

Under certain wind patterns or near coastlines, Ekman transport can lead to **Ekman Pumping**, where water is upwelled or downwelled depending on the wind direction. 

This process is particularily important for coastal ecosystems that rely on this upwelling (which is often seasonal) to bring nutrient-rich waters near the surface. 

These nutrients fuel the growth of tiny marine plants that support the entire food chain, upon which coastal economies often rely. 

These tiny plants are also responsible for most of the oxygen we breathe!

# Thermohaline Circulation

## Thermohaline Data

In [ ]:
# Monthly surface data over the period 2014 to 2016.
filename_theta = "surface_theta.nc"
url_theta = "https://osf.io/98ksr/download"

subset_theta = xr.open_dataset(pooch_load(url_theta, filename_theta))
subset_theta

## Thermohaline Data

In [ ]:
filename_salt = "surface_salt.nc"
url_salt = "https://osf.io/aufs2/download"

subset_salt = xr.open_dataset(pooch_load(url_salt, filename_salt))
subset_salt

## Plot Surface Temperature and Salinity

The ocean flows can be driven by density variations in addition to wind-driven circulation. 

One example of a density-driven flow is the thermohaline circulation.

Density in the ocean is influenced by two main factors: 

1. Salinity (higher salinity leads to greater density) and 
2. Temperature (lower temperature generally results in higher density),
3. Also, pressure affects density (higher pressure results in higher density), but it generally has a much smaller impact on ocean density than temperature and salinity. 

To develop a better understanding of how density varies across different regions, let's examine the average salinity and temperature at the ocean surface.

## Plot Surface Temperature and Salinity

In [ ]:
subset_theta = subset_theta.where(subset_theta != 0)
subset_salt = subset_salt.where(subset_salt != 0)
subset_theta = subset_theta.THETA
subset_salt = subset_salt.SALT

## Sea Surface Temperature

In [ ]:
# this is from cartopy https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(11, 12), dpi=100)
p = subset_theta.plot(vmin=0, cmap=cmocean.cm.thermal, 
                      cbar_kwargs={"shrink": 0.75, "orientation": "horizontal", "extend": "both",
                                   "pad": 0.05, "label": "degree C",}, ax=ax)
ax.coastlines(color="grey", lw=0.5)
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
ax.set_title("Sea Surface Temperature (2014-2016 mean)")
fig.tight_layout()

## Sea Surface Salinity

In [ ]:
# this is from cartopy https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(11, 12), dpi=100)  
p = subset_salt.plot(cmap=cmocean.cm.haline,
    vmin=30, cbar_kwargs={"shrink": 0.75,"orientation": "horizontal","extend": 
                          "both","pad": 0.05,"label": "psu",},ax=ax)
ax.coastlines(color="grey", lw=0.5)
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
ax.set_title("Sea Surface Salinity (2014-2016 mean)")
fig.tight_layout()

## Calculating Density from Salinity and Temperature

The equation relating ocean water density to other water properties is called the ***equation of state***.

It is a non-linear function of temperature, salinity, and pressure. 

This can be expressed as $$\rho=\rho(T,S,p)$$

Here we will show two ways to calculate the density. 

The first is a *linear approximation* to the equation of state.

## Linearized Equation of State

Here we take the linearized equation of state from equation 1.57 in Vallis' textbook ["*Atmospheric and Oceanic Fluid Dynamics*"](https://www.cambridge.org/core/books/atmospheric-and-oceanic-fluid-dynamics/41379BDDC4257CBE11143C466F6428A4)

$$ \rho=\rho_0[1-\beta_T(T-T_0)+\beta_S(S-S_0)+\beta_p(p-p_0)] $$

- $\rho_0\simeq 1027$ is a reference density
- $\beta_T \simeq 2*10^{-4}$/K is the thermal expansion coefficient
- $\beta_S \simeq 7.6*10^{-4}$/ppt is the haline contraction coefficient
- $\beta_p \simeq 4.4*10^{-10}$/Pa is the compressibility coefficient. 

The values with $_0$ are reference values, and here we use $T_0=283$K and $S_0=35$. 

Since surface pressure rarely changes by more than a few percent, let's assume that the pressure at the surface is equal to the reference pressure at every point ($\beta_p(p-p_0)=0$).

Let's now calculate a global map of surface density using this linear equation of state. Note that since we are using theta and salt *datasets*, our result will also be a dataset.

## Linearized Equation of State

In [ ]:
# Equation estimates (linearity assumption)
rho_linear = 1027 * (
    1 - 2e-4 * (subset_theta + 273.15 - 283) + 7.6e-4 * (subset_salt - 35)
)
rho_linear

## Linearized Density Plot

In [ ]:
# plot linearized density
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(11, 12), dpi=100
)  # this is from cartopy https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
p = rho_linear.plot(
    cmap=cmocean.cm.dense,
    vmin=1021,
    vmax=1029,
    cbar_kwargs={
        "shrink": 0.75,
        "orientation": "horizontal",
        "extend": "both",
        "pad": 0.05,
        "label": "kg/m$^3$",
    },
    ax=ax,
)
ax.coastlines(color="grey", lw=0.5)
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
ax.set_title("Surface density from linear equation (2014-2016 mean)")
fig.tight_layout()

## Full Nonlinear Equation of State

The full, non-linear equation of state is more complicated than the linear equation we just used. 

It contains dozens of equations which are impractical to code in this class. 

Fortunately packages exist to do this calculation!

Here we will compute surface density from the full nonlinear equation in `python` using the `gsw` package which is a Python implementation of the [Thermodynamic Equation of Seawater 2010 (TEOS-10)](https://teos-10.github.io/GSW-Python/)

## Full Nonlinear Equation of State

In [ ]:
## Non-linear equation
CT = gsw.CT_from_pt(
    subset_salt, subset_theta
)  # get conservative temperature from potential temperature
rho_nonlinear = gsw.rho(subset_salt, CT, 0)

## Full Nonlinear Equation - Plot

In [ ]:
# plot density from full nonlinear equation
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(11, 12), dpi=100
)  # this is from cartopy https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
p = rho_nonlinear.plot(
    cmap=cmocean.cm.dense,
    vmin=1021,
    vmax=1029,
    cbar_kwargs={
        "shrink": 0.75,
        "orientation": "horizontal",
        "extend": "both",
        "pad": 0.05,
        "label": "kg/m$^3$",
    },
    ax=ax,
)
ax.coastlines(color="grey", lw=0.5)
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
ax.set_title("Surface density from nonlinear equation (2014-2016 mean)")
fig.tight_layout()

## Difference between Linear and Non-Linear

In [ ]:
# plot difference between linear and non-linear equations of state
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(11, 12), dpi=100
)  # this is from cartopy https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
p = (rho_linear - rho_nonlinear).plot(
    cmap="coolwarm",
    vmin=-3,
    vmax=3,
    cbar_kwargs={
        "shrink": 0.75,
        "orientation": "horizontal",
        "extend": "both",
        "pad": 0.05,
        "label": "kg/m$^3$",
    },
    ax=ax,
)
ax.coastlines(color="grey", lw=0.5)
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.add_feature(cart.feature.LAND, zorder=100, edgecolor="k")
ax.set_title("Linear minus non-linear equation of state (2014-2016 mean)")
fig.tight_layout()

## Difference between Linear and Non-Linear

Upon comparing the two equations of state, we observe that they are generally similar, but certain differences arise. 

These differences stem from the nonlinearity of the equation of state, where the haline contraction coefficient and thermal expansion coefficient are not constant as assumed in our linear equation of state.

Irrespective of the method used to calculate density, we notice the presence of horizontal density variations (gradients) at the ocean surface.

For instance, seawater tends to be less dense in the subtropics and denser near the poles.

These density differences play a crucial role in driving ocean currents, as we discussed in the slides.

These findings emphasize the significant density gradients in the ocean, which shape oceanic circulation patterns. 

The nonlinearity in the equation of state contributes to these density variations, which in turn also influences the movement of water masses and the formation of currents.

## Questions?

## See you next lecture!